In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module="matplotlib")
warnings.filterwarnings('ignore')


from sklearn.metrics import roc_curve, auc, precision_recall_curve, f1_score


def roc_auc_plot(y_test, y_scores):
    # ROC Curve and AUC
    fpr, tpr, roc_thresholds = roc_curve(y_test, y_scores)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(10, 5))

    # Plot ROC
    plt.subplot(1, 2, 1)
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--', label='Random Guess')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc='lower right')
    plt.grid(True)

    # Precision-Recall Curve
    precision, recall, pr_thresholds = precision_recall_curve(y_test, y_scores)
    pr_auc = auc(recall, precision)

    plt.subplot(1, 2, 2)
    plt.plot(recall, precision, label='Precision-Recall curve (area = %0.2f)' % pr_auc)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend(loc='lower left')
    plt.grid(True)

    plt.tight_layout()
    plt.show()



def show_metrics(y_test, y_prob, y_pred):

    print("Classification Report:\n", classification_report(y_test, y_pred))
    print("ROC AUC Score:", roc_auc_score(y_test, y_prob))
    roc_auc_plot(y_test, y_prob)

    # Confusion Matrix
    plt.figure(figsize=(6, 4))
    sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title('Confusion Matrix')
    plt.show()

    
    
def build_single_layer_neural_network_tf(n):
    
    hidden_features = n
    return tf.keras.Sequential([tf.keras.layers.Dense(128, activation='relu',input_shape=(n,)),
                                tf.keras.layers.Dense(128, activation='relu'),
                                tf.keras.layers.Dense(64, activation='relu'),
                                tf.keras.layers.Dense(1, activation='sigmoid')
                               ])

    
def train_nn(X_train, X_test, y_train, y_test):
    predictions = []
    probs = []
    
    single_layer_neural_network_tf = build_single_layer_neural_network_tf(X_train.shape[1])

    single_layer_neural_network_tf, _, _ = train_tensorflow(X_train,
                                                            y_train,
                                                            single_layer_neural_network_tf,
                                                            loss = tf.keras.losses.BinaryCrossentropy(),
                                                            optimizer = 'adam',
                                                            metrics = ['accuracy'],
                                                            epochs=50,
                                                            batch_size=100,
                                                            validation_data=(X_test, y_test),
                                                            verbose=0,
                                                            plots=True)

    y_pred, y_prob = predict_tf(single_layer_neural_network_tf, X_test, verbose=1)
    predictions.append(y_pred)
    probs.append(y_prob)
        
    correct_predictions = (y_pred == y_test).sum()
    total_samples = y_test.shape[0]
    accuracy = correct_predictions / total_samples
    print(f"{accuracy = }")
    show_metrics(y_test, y_prob, y_pred)

predictions, probs = train_nn(X_train, X_test, y_train, y_test)